In [56]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [57]:
dataset_path = r'C:\Users\krish\Downloads\archive (3)\traffic_Data'
''' NOTE: I put all the test data all in subdirectories just like how DATA folder is structured , also a bunch of classes had no values in TEST so i copied some over from DATA'''

In [58]:
def load_dataset(dataset_path, subset):
    return image_dataset_from_directory(
        os.path.join(dataset_path, subset),
        image_size=(32, 32),
        batch_size=32,
        shuffle=True
    )

In [59]:
# Load datasets
OGtrain_dataset = load_dataset(dataset_path, "DATA")
test_dataset = load_dataset(dataset_path, "TEST")

Found 4174 files belonging to 58 classes.
Found 2009 files belonging to 58 classes.


In [60]:
# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25), 
    layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25), 
    layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    layers.Flatten(),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.15), 
    layers.Dense(58, activation='softmax')  # Adjusted for 58 classes
])

In [61]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [62]:
#model.fit(train_dataset, epochs=50)

In [63]:
# Split the train dataset into train and validation
# Take 20% of the train dataset as validation
val_size = int(0.2 * len(OGtrain_dataset))  # 20% for validation
train_size = len(OGtrain_dataset) - val_size

# Shuffle the train dataset and split into train and validation
train_dataset = OGtrain_dataset.take(train_size)  # Training dataset
val_dataset = OGtrain_dataset.skip(train_size)  # Validation dataset

In [64]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)  # Exponentially decay the learning rate after epoch 10
    
model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=[early_stopping, LearningRateScheduler(lr_scheduler)])

Epoch 1/50
105/105 [==============================] - 6s 45ms/step - loss: 5.4987 - accuracy: 0.2101 - val_loss: 2.6691 - val_accuracy: 0.4975 - lr: 0.0010
Epoch 2/50
105/105 [==============================] - 5s 42ms/step - loss: 2.2146 - accuracy: 0.5065 - val_loss: 1.5084 - val_accuracy: 0.7273 - lr: 0.0010
Epoch 3/50
105/105 [==============================] - 5s 43ms/step - loss: 1.4901 - accuracy: 0.6875 - val_loss: 1.1334 - val_accuracy: 0.7899 - lr: 0.0010
Epoch 4/50
105/105 [==============================] - 5s 43ms/step - loss: 1.1546 - accuracy: 0.7720 - val_loss: 0.8698 - val_accuracy: 0.8526 - lr: 0.0010
Epoch 5/50
105/105 [==============================] - 5s 43ms/step - loss: 0.8953 - accuracy: 0.8363 - val_loss: 0.7092 - val_accuracy: 0.9017 - lr: 0.0010
Epoch 6/50
105/105 [==============================] - 5s 43ms/step - loss: 0.7651 - accuracy: 0.8726 - val_loss: 0.6600 - val_accuracy: 0.9128 - lr: 0.0010
Epoch 7/50
105/105 [==============================] - 5s 42ms/st

In [65]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")

63/63 [==============================] - 1s 12ms/step - loss: 1.7908 - accuracy: 0.7203
Test Accuracy: 0.7203


In [66]:
def visualize_filters_and_pools(model, image):
    layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name or 'pool' in layer.name]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(image.reshape(1, 32, 32, 3))
    
    for i, activation in enumerate(activations):
        num_filters = activation.shape[-1]
        size = activation.shape[1]
        fig, axes = plt.subplots(1, min(num_filters, 6), figsize=(15, 5))
        for j in range(min(num_filters, 6)):
            ax = axes[j]
            ax.imshow(activation[0, :, :, j], cmap='viridis')
            ax.axis('off')
        plt.show()